<a href="https://colab.research.google.com/github/namita0210/german_to_english_machine_translation/blob/main/German_to_English_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import string
from numpy import array
from pickle import dump
from unicodedata import normalize
from pickle import load
from pickle import dump
from numpy.random import rand
from numpy.random import shuffle

from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
#from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint

In [2]:
from google.colab import drive
drive.mount('/content/drive')
data = '/content/drive/MyDrive/deu.txt'

Mounted at /content/drive


In [3]:
#function to load the file and preserve the unicode german characters
def load_file(filename):
  file = open(filename , 'r', encoding='utf-8')
  text = file.read()
  file.close()
  return text

In [4]:
text=load_file(data)
print(text[:99])

Hi.	Hallo!
Hi.	Grüß Gott!
Run!	Lauf!
Wow!	Potzdonner!
Wow!	Donnerwetter!
Fire!	Feuer!
Help!	Hilfe!



In [5]:
#split the text by phrases
def to_phrase(doc):
  lines = doc.strip().split('\n')
  phrases =[line.split('\t') for line in lines]
  return phrases

In [6]:
phrases=to_phrase(text)
print(phrases[:3])
print(type(phrases))

[['Hi.', 'Hallo!'], ['Hi.', 'Grüß Gott!'], ['Run!', 'Lauf!']]
<class 'list'>


In [7]:
string.printable

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [8]:
#clean a list of lines
def clean_phrase(lines):
  cleaned = []
  re_print = re.compile('[^%s]' % re.escape(string.printable)) #remove non-printable characters
  table = str.maketrans('','',string.punctuation)
  for pair in lines:
    clean_pair=[]
    for p in pair:
      p = normalize('NFD',p).encode('ascii','ignore')
      p = p.decode('UTF-8')
      p = p.split()
      p = [word.lower() for word in p]
      p = [word.translate(table) for word in p] # remove punctuation
      p = [re_print.sub('',w) for w in p]
      p = [word for word in p if word.isalpha()]
    clean_pair.append(''.join(p))
  cleaned.append(clean_pair)
  return array(cleaned)

In [9]:
# save a list of clean sentences to file
def save_clean_data(sentences, filename):
  dump(sentences, open(filename, 'wb'))
  print('Saved: %s' % filename)

In [10]:
doc = load_file(filename=data)
pairs = to_phrase(doc)
clean_pairs = clean_phrase(pairs)

In [11]:
save_clean_data(clean_pairs, 'english-german.pkl') # Run Only Once.

Saved: english-german.pkl


In [12]:
#train-test-split

def load_clean_sentences(filename):
 return load(open(filename, 'rb'))

raw_dataset = load_clean_sentences('english-german.pkl')

In [13]:
n_sentences = 10000
dataset = raw_dataset[:n_sentences, :]
shuffle(dataset)
train, test = dataset[:9000], dataset[9000:]
save_clean_data(dataset, 'english-german-both.pkl')
save_clean_data(train, 'english-german-train.pkl')
save_clean_data(test, 'english-german-test.pkl')

Saved: english-german-both.pkl
Saved: english-german-train.pkl
Saved: english-german-test.pkl


In [14]:
#Train Neural Translation model
#Start by loading the train - test data stored in pkl files
# Step 7 : Load the pkl files

dataset = load_clean_sentences('english-german-both.pkl')
train = load_clean_sentences('english-german-train.pkl')
test = load_clean_sentences('english-german-test.pkl')

In [15]:
#We can use the Keras Tokenize class to map words to integers, as needed for modeling.
#We will use separate tokenizer for the English sequences and the German sequences.
#The function below-named create_tokenizer() will train a tokenizer on a list of phrases
#Step 8 : Create tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [16]:
# Find the length of longest sequence in the list of phrases
# max sentence length
def max_length(lines):
 return max(len(line.split()) for line in lines)